# Sujet 2 : Évaluation du Générateur Congruentiel Linéaire (GCL) RANDU

Composantes du groupe : Moyano Axel, Bonnassie Enzo
Groupe 2A BUT2

## Objectif
Ce projet a pour but d'évaluer la qualité statistique du générateur congruentiel linéaire (GCL) RANDU, défini par la récurrence :
\[ x_{n+1} \equiv (65539 \times x_n) \mod 2^{31} \]
avec une valeur initiale \( x_0 = 12345 \).

Les nombres générés sont ensuite normalisés dans l'intervalle \([0, 1]\) pour être analysés. Nous étudierons ses performances pour trois tailles d'échantillons :
- **30 000 données**
- **90 000 données**
- **300 000 données**

L'objectif est de vérifier si ce générateur **passe des tests statistiques rigoureux** :
- **Test du khi-deux** (seuil de confiance de 1%) pour évaluer l'uniformité de la distribution.
- **Test spectral 2D** sur des doublets \((x_i, x_{i+1})\) pour détecter des corrélations entre paires consécutives.
- **Test spectral 3D** sur des triplets \((x_i, x_{i+1}, x_{i+2})\) pour analyser la répartition des points dans un espace tridimensionnel.

Cette étude permettra de déterminer si le **GCL RANDU** est adapté pour des applications nécessitant une **bonne qualité aléatoire**, ou s'il présente des **faiblesses** (comme des corrélations ou une mauvaise uniformité), le rendant inapproprié pour des usages critiques (simulations, cryptographie, etc.).

---

## Présentation
Le **GCL RANDU** est un exemple classique de **générateur pseudo-aléatoire simple**, mais il est connu pour ses **limites statistiques**, notamment des **corrélations entre les nombres générés**.

En le soumettant à ces tests, nous pourrons :
- **Quantifier ses défauts**.
- Comprendre pourquoi il est aujourd’hui **déconseillé** pour des applications exigeantes.

Les résultats seront **visualisés sous forme de graphiques** (2D et 3D) et analysés pour conclure sur sa robustesse. Ce travail s’inscrit dans une démarche d’**évaluation critique des algorithmes de génération aléatoire**, essentielle en informatique et en modélisation.


In [ ]:

def normaliser(liste):
    """
    Normalise une liste de valeurs dans l'intervalle [0, 1].

    Cette fonction prend une liste de nombres et retourne une nouvelle liste
    où chaque élément est normalisé entre 0 et 1 en.
    Args:
        liste (list): Une liste de nombres (entiers ou flottants).

    Returns:
        list: Une nouvelle liste contenant les valeurs normalisées entre 0 et 1.

    Raises:
        ValueError: Si la liste est vide ou si tous les éléments sont identiques
                   (ce qui entraînerait une division par zéro).
    """
    minimum = min(liste)
    maximum = max(liste)

    # Gestion des cas particuliers
    if minimum == maximum:
        raise ValueError("Tous les éléments de la liste sont identiques. Impossible de normaliser.")

    newlist = []
    for i in liste:
        newlist.append((i - minimum) / (maximum - minimum))
    return newlist




def GCL_RANDU(nbElement):
    """
    Génère une séquence de nombres pseudo-aléatoires en utilisant l'algorithme RANDU.

    Cet algorithme utilise la récurrence linéaire congruentielle (GCL) suivante :
    \[
    U_{n+1} = (65539 \times U_n) \mod 2^{31}
    \]
    où \( U_0 = 12345 \) est la valeur initiale.

    Les nombres générés sont ensuite normalisés entre 0 et 1 grace a la fopnction normaliser

    Args:
        nbElement (int): Le nombre d'éléments à générer (y compris la valeur initiale).

    Returns:
        list: Une liste de nombres pseudo-aléatoires normalisés entre 0 et 1.
    """
    Un0 = 12345
    liste = []
    liste.append(Un0)

    for i in range(nbElement - 1):  # On commence déjà avec Un0
        UnPlus1 = (65539 * liste[i]) % (2**31)
        liste.append(UnPlus1)

    return normaliser(liste)


##Test d'affichage
liste = []
liste = GCL_RANDU(10)
print(liste)

In [ ]:
import numpy as np
from scipy.stats import chi2_contingency, chi2

def test_khi_deux(donnees, nb_classes=10, seuil_confiance=0.01):

    # Diviser les données en classes
    classes = np.linspace(0, 1, nb_classes + 1)
    observed, _ = np.histogram(donnees, bins=classes)

    # Distribution uniforme attendue
    expected = np.full(nb_classes, len(donnees) / nb_classes)

    # Calcul du khi-deux
    chi2_stat, p_value, _, _ = chi2_contingency([observed, expected])

    # Comparaison avec le seuil de confiance
    passe_le_test = p_value > seuil_confiance

    return passe_le_test, p_value

# Exemple d'utilisation avec 30 000 données
liste = []
liste = GCL_RANDU(30000)
passe_30k, p_value_30k = test_khi_deux(liste)
print(f"Test pour 30 000 données : {'Passe' if passe_30k else 'Échoue'} (p-value = {p_value_30k:.4f})")

# Exemple d'utilisation avec 90 000 données
liste = GCL_RANDU(90000)
passe_90k, p_value_90k = test_khi_deux(liste)
print(f"Test pour 90 000 données : {'Passe' if passe_90k else 'Échoue'} (p-value = {p_value_90k:.4f})")

# Exemple d'utilisation avec 300 000 données
liste = GCL_RANDU(300000)
passe_300k, p_value_300k = test_khi_deux(liste)
print(f"Test pour 300 000 données : {'Passe' if passe_300k else 'Échoue'} (p-value = {p_value_300k:.4f})")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def normaliser(liste):
    minimum = min(liste)
    maximum = max(liste)
    newlist = []
    for i in liste:
        newlist.append((i - minimum) / (maximum - minimum))
    return newlist

def GCL_RANDU(nbElement):
    liste = []
    Un0 = 12345
    liste.append(Un0)
    for i in range(nbElement - 1):  # On commence déjà avec Un0
        UnPlus1 = (65539 * liste[i]) % (2**31)
        liste.append(UnPlus1)
    return normaliser(liste)

def test_spectral_2d(donnees):
    """
    Réalise un test spectral 2D sur les doublets de données.

    Args:
        donnees (list): Liste des données générées (normalisées entre 0 et 1).

    Returns:
        float: Distance minimale entre deux doublets consécutifs.
    """
    # Regrouper les données en doublets
    doublets = [(donnees[i], donnees[i+1]) for i in range(len(donnees)-1)]

    # Calculer les distances entre doublets consécutifs
    distances = []
    for i in range(len(doublets)-1):
        x1, y1 = doublets[i]
        x2, y2 = doublets[i+1]
        distance = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
        distances.append(distance)

    # Distance minimale
    distance_min = min(distances)

    return doublets, distance_min

def afficher_doublets(doublets, taille):
    """
    Affiche les doublets dans un plan 2D.

    Args:
        doublets (list): Liste des doublets.
        taille (int): Taille des données (30k, 90k, 300k).
    """
    x, y = zip(*doublets)
    plt.figure(figsize=(8, 8))
    plt.scatter(x, y, s=1, alpha=0.5)
    plt.title(f"Test spectral 2D - {taille} données")
    plt.xlabel("x_i")
    plt.ylabel("x_{i+1}")
    plt.xlim(0, 1)
    plt.ylim(0, 1)
    plt.grid(True)
    plt.show()

# Test pour 30 000 données
liste_30k = GCL_RANDU(30000)
doublets_30k, distance_min_30k = test_spectral_2d(liste_30k)
print(f"Distance minimale pour 30 000 données : {distance_min_30k:.6f}")
afficher_doublets(doublets_30k, 30000)

# Test pour 90 000 données
liste_90k = GCL_RANDU(90000)
doublets_90k, distance_min_90k = test_spectral_2d(liste_90k)
print(f"Distance minimale pour 90 000 données : {distance_min_90k:.6f}")
afficher_doublets(doublets_90k, 90000)

# Test pour 300 000 données
liste_300k = GCL_RANDU(300000)
doublets_300k, distance_min_300k = test_spectral_2d(liste_300k)
print(f"Distance minimale pour 300 000 données : {distance_min_300k:.6f}")
afficher_doublets(doublets_300k, 300000)


In [ ]:
import numpy as np
import plotly.graph_objects as go

def normaliser(liste):
    minimum = min(liste)
    maximum = max(liste)
    newlist = []
    for i in liste:
        newlist.append((i - minimum) / (maximum - minimum))
    return newlist

def GCL_RANDU(nbElement):
    liste = []
    Un0 = 12345
    liste.append(Un0)
    for i in range(nbElement - 1):  # On commence déjà avec Un0
        UnPlus1 = (65539 * liste[i]) % (2**31)
        liste.append(UnPlus1)
    return normaliser(liste)

def test_spectral_3d(donnees):
    """
    Réalise un test spectral 3D sur les triplets de données.

    Args:
        donnees (list): Liste des données générées (normalisées entre 0 et 1).

    Returns:
        list: Liste des triplets.
        float: Distance minimale entre deux triplets consécutifs.
    """
    # Regrouper les données en triplets
    triplets = [(donnees[i], donnees[i+1], donnees[i+2]) for i in range(len(donnees)-2)]

    # Calculer les distances entre triplets consécutifs
    distances = []
    for i in range(len(triplets)-1):
        x1, y1, z1 = triplets[i]
        x2, y2, z2 = triplets[i+1]
        distance = np.sqrt((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2)
        distances.append(distance)

    # Distance minimale
    distance_min = min(distances)

    return triplets, distance_min

def afficher_3d_interactif(triplets, taille):
    """
    Affiche les triplets en 3D de manière interactive avec Plotly.

    Args:
        triplets (list): Liste des triplets.
        taille (int): Taille des données (30k, 90k, 300k).
    """
    x, y, z = zip(*triplets)

    fig = go.Figure(data=[go.Scatter3d(
        x=x,
        y=y,
        z=z,
        mode='markers',
        marker=dict(
            size=2,
            opacity=0.3,
        )
    )])

    fig.update_layout(
        title=f"Test spectral 3D - {taille} données",
        scene=dict(
            xaxis_title='x_i',
            yaxis_title='x_{i+1}',
            zaxis_title='x_{i+2}',
            xaxis=dict(range=[0, 1]),
            yaxis=dict(range=[0, 1]),
            zaxis=dict(range=[0, 1]),
        ),
        margin=dict(l=0, r=0, b=0, t=30),
    )

    fig.show()

# Test pour 30 000 données
liste_30k = GCL_RANDU(30000)
triplets_30k, distance_min_30k = test_spectral_3d(liste_30k)
print(f"Distance minimale pour 30 000 données : {distance_min_30k:.6f}")
##afficher_3d_interactif(triplets_30k, 30000)

# Test pour 90 000 données
liste_90k = GCL_RANDU(90000)
triplets_90k, distance_min_90k = test_spectral_3d(liste_90k)
print(f"Distance minimale pour 90 000 données : {distance_min_90k:.6f}")
##afficher_3d_interactif(triplets_90k, 90000)

# Test pour 300 000 données
liste_300k = GCL_RANDU(300000)
triplets_300k, distance_min_300k = test_spectral_3d(liste_300k)
print(f"Distance minimale pour 300 000 données : {distance_min_300k:.6f}")
##afficher_3d_interactif(triplets_300k, 300000)
